In [15]:
import polars as pl
import spice

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get DUNE_KEY from .env and set it as DUNE_API_KEY
dune_key = os.getenv('DUNE_KEY')
os.environ['DUNE_API_KEY'] = dune_key


In [17]:
df = spice.query(4481601)

loading result from cache


In [ ]:
df.head()

In [18]:
df.sort(pl.col('evt_block_number'))

evt_tx_from,evt_tx_to,value,evt_block_number
str,str,f64,f64
"""0x7798ba9512b5a684c12e31518923…","""0x767e095f6549050b4e9a3bcce18a…",1e8,2.3940784e7
"""0x7798ba9512b5a684c12e31518923…","""0x767e095f6549050b4e9a3bcce18a…",1e8,2.3940784e7
"""0x7798ba9512b5a684c12e31518923…","""0x767e095f6549050b4e9a3bcce18a…",1.691e6,2.3940784e7
"""0x7798ba9512b5a684c12e31518923…","""0x767e095f6549050b4e9a3bcce18a…",800000.0,2.3940784e7
"""0x7798ba9512b5a684c12e31518923…","""0x767e095f6549050b4e9a3bcce18a…",550000.0,2.3940784e7
…,…,…,…
"""0x0278778593a7b73918f1906c7f55…","""0x1111111254eeb25477b68fb85ed9…",22.652848,2.4223285e7
"""0x0278778593a7b73918f1906c7f55…","""0x1111111254eeb25477b68fb85ed9…",2265.284754,2.4223285e7
"""0x7b533d27235b7a5df992edb42fdc…","""0x6131b5fae19ea4f9d964eac0408e…",54652.811263,2.4223285e7


In [19]:

# Create a DataFrame with two separate rows for each transaction
df_expanded = pl.concat([
    df.select(
        pl.col('evt_tx_from').alias('address'),
        pl.col('value').cast(pl.Float64) * -1,  # Negative for outgoing
        pl.col('evt_block_number')
    ),
    df.select(
        pl.col('evt_tx_to').alias('address'),
        pl.col('value').cast(pl.Float64),  # Positive for incoming
        pl.col('evt_block_number')
    )
])

# Group by address and sum the values to get final balances
final_balances = df_expanded.group_by('address').agg(
    pl.col('value').sum().alias('final_balance')
)

# Sort by final balance in descending order (optional)
final_balances = final_balances.sort('final_balance', descending=True)

print(final_balances)

# Optional: Verify that the total sum is zero (if it should be)
total_sum = df_expanded['value'].sum()
print(f"\nTotal sum (should be close to 0): {total_sum}")


shape: (22_546, 2)
┌─────────────────────────────────┬───────────────┐
│ address                         ┆ final_balance │
│ ---                             ┆ ---           │
│ str                             ┆ f64           │
╞═════════════════════════════════╪═══════════════╡
│ 0x6131b5fae19ea4f9d964eac0408e… ┆ 6.3993e9      │
│ 0x3fc91a3afd70395cd496c647d5a6… ┆ 3.9015e9      │
│ 0x743f2f29cdd66242fb27d292ab2c… ┆ 2.0683e9      │
│ 0x03a520b32c04bf3beef7beb72e91… ┆ 1.4764e9      │
│ 0x6b2c0c7be2048daa9b5527982c29… ┆ 1.4217e9      │
│ …                               ┆ …             │
│ 0x1a72876ebe781e42ab2ee4278b53… ┆ -2.9751e8     │
│ 0xb99005b2bf1ae43f15ba262fdf07… ┆ -3.1410e8     │
│ 0x7798ba9512b5a684c12e31518923… ┆ -1.2010e9     │
│ 0xeea4b70c003d3cce9069b2ec34f4… ┆ -1.2084e9     │
│ 0x2fd1d8eaa6ed1b424c2d03ce5ead… ┆ -1.2257e9     │
└─────────────────────────────────┴───────────────┘

Total sum (should be close to 0): 0.0
